## FDTree Class

In [13]:
import numpy as np
import pickle
from scipy.sparse import *
import shelve
import pickle

class DecisionTreeModel:
    def __init__(self, source, depth_threshold=10, plambda=7, MSP_item=200):

        x = find(source)
        itemset = x[0]
        userset = x[1]
        self.rU = {}
        
        #### Generate rI, rU ####
        print("rU Generation start:")

        num_ratings = len(userset)
        i = 0
        for itemid, userid in zip(itemset, userset):
            n_i = int(userid/24)  
            if i%100000 == 0:
                print("%.2f%%" %(100*i/num_ratings))
            i += 1
            if n_i in self.rU:
                self.rU[n_i].setdefault(userid, {})[itemid] = int(source[itemid, userid])
            else:
                self.rU.setdefault(n_i, {userid: {itemid: int(source[itemid, userid])}})
                
                
        print("rU Generation DONE")
        
        print("bias, sum_cur_t, sum_2_cur_t Generation start:")
        
        self.rI = list(set(source.nonzero()[0]))
        self.biasU = np.zeros(self.sMatrix.shape[1])
        self.sum_cur_t = np.zeros(self.real_item_num)
        self.sum_2_cur_t = np.zeros(self.real_item_num)
        self.sum_cntt = np.zeros(self.real_item_num)
        i = 0
        for userid in self.tree:
            if i % 5000 == 0:
                print("%.2f%%" % (100 * i / (0.75 * 480189)))
            i += 1

            self.biasU[userid] = (self.sMatrix.getcol(userid).sum() \
                                     + self.plambda * self.global_mean) /   \
                                 (self.plambda + self.sMatrix.getcol(userid).getnnz())
            user_all_rating_id = self.sMatrix.getcol(userid).nonzero()[0]
            # print('user_all_rating_id ', user_all_rating_id[:])
            user_all_rating = find(self.sMatrix.getcol(userid))[2]
            self.sum_cur_t[user_all_rating_id[:]] += user_all_rating[:] - self.biasU[userid]
            self.sum_2_cur_t[user_all_rating_id[:]] += (user_all_rating[:] - self.biasU[userid]) ** 2
            self.sum_cntt[user_all_rating_id[:]] += 1
            
        print("bias, sum_cur_t, sum_2_cur_t Generation DONE")
        
        print("Initiation DONE!")

## Step 1: Input Dataset

In [14]:
from scipy.sparse import load_npz

In [15]:
############### Load Data ##################
rating_matrix_csc = load_npz('netflix/sparse_matrix_100%.npz').tocsc()
print("file load DONE")
############################################

file load DONE


In [16]:
''' Save to file 'tree.pkl' '''
start = 0
end = int(rating_matrix_csc.shape[1] * 0.75)

In [18]:
dt_model = DecisionTreeModel(rating_matrix_csc[0:100,0:1000])

rU Generation start:
0.00%
rU Generation DONE
Initiation DONE!


In [ ]:
del rating_matrix_csc

In [21]:
for dictname in dt_model.rU:
    d = shelve.open('./rU_data/'+str(dictname), protocol = pickle.HIGHEST_PROTOCOL)
    d['content'] = dt_model.rU[dictname]
    d.close()
print("DONE!")

DONE!


In [25]:
dictt = {}
for dictname in range(151):
    d = shelve.open('./rU_data/'+str(dictname), protocol = pickle.HIGHEST_PROTOCOL)
    dictt.update(d['content'])
    d.close()

In [28]:
print(len(dictt.keys()))

360141


In [ ]:
# import klepto
# Tree = klepto.archives.dir_archive('treeFile', cached=True, serialized=True)

# Tree['biasU'] = dtmodel_realdata.biasU
# Tree['sum_cur_t'] = dtmodel_realdata.sum_cur_t
# Tree['sum_2_cur_t'] = dtmodel_realdata.sum_2_cur_t
# Tree['sum_cntt'] = dtmodel_realdata.sum_cntt
# Tree['rI'] = dtmodel_realdata.rI

# Tree.dump()
# Tree.clear()

# save_npz('netflix/biasUM', dtmodel_realdata.biasUM.tocsr())